In [ ]:
from google.colab import drive
drive.mount('/content/drive/')

In [ ]:
import pickle
import gc
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

import numpy as np
import pandas as pd
from keras.models import Model
from keras.layers import Input, Dense, Flatten, Concatenate, concatenate
from keras.layers import Dense, Activation, Conv1D
from keras import optimizers
from keras.optimizers import Adam
import keras
from keras.models import Sequential
from keras import regularizers

from sklearn.metrics import f1_score

from sklearn.metrics import roc_curve
from matplotlib import pyplot
import os

In [3]:
DataPath = '/content/drive/My Drive/apps/stress detection/data/wrist'

In [ ]:
#!ls '/content/drive/My Drive/apps/stress detection/data/wrist'

In [4]:
# specify file names
fileName_target = DataPath + '/' + 'data_wrist_groundTruth_2classes_stress_5secWindow.pkl'
fileName_wrist_BVP = DataPath + '/' + 'data_wrist_BVP_5secWindow,pkl'
fileName_wrist_EDA = DataPath + '/' + 'data_wrist_EDA_5secWindow,pkl'
fileName_wrist_TEMP = DataPath + '/' + 'data_wrist_TEMP_5secWindow,pkl'

In [5]:
# read data
with open(fileName_target, 'rb') as f:
   data_targets = pickle.load(f)
with open(fileName_wrist_BVP,'rb') as f:
   data_wrist_BVP = pickle.load(f)
with open(fileName_wrist_EDA,'rb') as f:
   data_wrist_EDA = pickle.load(f)
with open(fileName_wrist_TEMP,'rb') as f:
   data_wrist_TEMP = pickle.load(f)


In [ ]:
print ('data_targets shape:', data_targets.shape)
print ('data_wrist_EDA shape:', data_wrist_EDA.shape)
print ('data_wrist BVP shape:', data_wrist_BVP.shape)
print ('data_wrist TEMP shape:', data_wrist_TEMP.shape)

In [ ]:
data = np.concatenate([data_wrist_BVP, data_wrist_EDA, data_wrist_TEMP], axis=1)
print ('data shape:', data.shape)
del data_wrist_EDA, data_wrist_BVP, data_wrist_TEMP
gc.collect()

In [ ]:
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data)
print ('data_scaled shape:', data_scaled.shape)
del data
gc.collect()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data_scaled, data_targets, test_size=0.30, random_state=42)
print ("X_train size:", X_train.shape, "Y_train size", len(y_train))
print ("X_test size:", X_test.shape, "Y_test size", len(y_test))

In [10]:
wsize_bvp = 320
wsize_eda = 20
wsize_temp = 20
wsize1 = wsize_bvp
wsize2 = wsize_bvp + wsize_eda
wsize3 = wsize_bvp + wsize_eda + wsize_temp

In [11]:
x_tr_1 = X_train[:,0:wsize1     ]
x_tr_2 = X_train[:,wsize1:wsize2]
x_tr_3 = X_train[:,wsize2:wsize3]

x_tst_1 = X_test[:,0:wsize1     ]
x_tst_2 = X_test[:,wsize1:wsize2]
x_tst_3 = X_test[:,wsize2:wsize3]

In [12]:
def binary_decision(prob):
  dec=[]
  for k in range(len(prob)):
    if prob[k]>=.5:
      d = 1
    else:
      d = 0
    dec.append(d)
  return dec

def accuracy(y_d, y_e):
  numErr=0
  for k in range(len(y_d)):
    if y_d[k]!=y_e[k]:
      numErr+=1
  return 1.0 - numErr/len(y_d)

In [13]:
# mlp neural network model for stress detection using signals from sensors BVP, EDA, and TEMP
def model_mlp_bvp_eda_temp():
   inputs_1 = Input(shape=(wsize_bvp,))
   inputs_2 = Input(shape=(wsize_eda,))
   inputs_3 = Input(shape=(wsize_temp,))

   x1 = Dense(64, kernel_regularizer=regularizers.l2(0.01), activation='relu')(inputs_1)
   x1 = Dense(32, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x1)
   x = concatenate([x1, inputs_2, inputs_3])
   x = Dense(32, kernel_regularizer=regularizers.l2(0.01), activation='relu')(x)
   x = Dense(16, activation='relu')(x)
   y = Dense(8, activation='relu')(x)
   out = Dense(1, activation = 'sigmoid')(y)

   model2r = Model([inputs_1, inputs_2, inputs_3], out)
   return model2r


In [ ]:
# 10-fold
seed = 7
np.random.seed(seed)

n_folds = 10
kfold = KFold(n_folds, True, 1)

y_test_pred=np.zeros(y_test.shape)

y_preds_valid = []
y_trues_valid = []

k=0
for train_ix, test_ix in kfold.split(x_tr_1):
#  model = model_mlp_bvp_eda_temp_xyzAcc()
   model = model_mlp_bvp_eda_temp()
   model.compile(optimizer = Adam(lr=0.001, clipnorm=1.), loss="binary_crossentropy", metrics=['accuracy'])
    
   print ("train model", k)
   k = k+1
   Xfold_train = [x_tr_1[train_ix,:], x_tr_2[train_ix,:], x_tr_3[train_ix,:] ]
   Yfold_train = y_train[train_ix]
   Xfold_valid =  [x_tr_1[test_ix,:],  x_tr_2[test_ix,:], x_tr_3[test_ix,:] ]
   Yfold_valid = y_train[test_ix]
   history = model.fit(Xfold_train, Yfold_train, epochs=50, batch_size=40, validation_data=(Xfold_valid, Yfold_valid))

   # accumulate predictions for validation and test
   y_test_pred += model.predict([x_tst_1, x_tst_2, x_tst_3])

   y_preds_valid.append( model.predict( Xfold_valid ) )
   y_trues_valid.append( Yfold_valid )

y_test_pred = y_test_pred/n_folds

In [ ]:
print (y_test_pred.shape)

In [ ]:
# calculate accuracy
y_pred_dec = binary_decision(y_test_pred)
y_test_accuracy = accuracy(y_test, y_pred_dec)
print ("accuracy_test:", y_test_accuracy)

In [ ]:
# calculate F1 score
y_test_dec = y_test
print('f1 score:', f1_score(y_test_dec, y_pred_dec) )

In [ ]:
# plot no skill roc curve
pyplot.plot([0, 1], [0, 1], linestyle='--', label='No Skill')
# calculate roc curve for model
fpr, tpr, _ = roc_curve(y_test, y_test_pred)
# plot model roc curve
pyplot.plot(fpr, tpr, marker='.', label='1D Convolutional Neural Network')
# axis labelsona
pyplot.xlabel('False Positive Rate')
pyplot.ylabel('True Positive Rate')
# show the legend
pyplot.legend()
# show the plot
pyplot.show()